#### LangChain Essentials Course

# LangChains Expression Language

LangChain is one of the most popular open source libraries for AI Engineers. It's goal is to abstract away the complexity in building AI software, provide easy-to-use building blocks, and make it easier when switching between AI service providers.

In this example, we will introduce LangChain's Expression Langauge (LCEL), abstracting a full chain and understanding how it will work. We'll provide examples for both OpenAI's `gpt-4o-mini` *and* Meta's `llama3.2` via Ollama!

## Choosing your Model

This example is split into two versions - The [Ollama version](), allowing us to run our LLM locally without needing any external services or API keys. The [OpenAI version](https://github.com/aurelio-labs/agents-course/blob/main/04-langchain-ecosystem/01-langchain-essentials/01-langchain-intro-openai.ipynb) uses the OpenAI API and requires an OpenAI API key.

## Initializing Llama 3.2

We start by initializing the 1B parameter Llama 3.2 model, fine-tuned for instruction following. We pull the model from Ollama by switching to our terminal and executing:

ollama pull llama3.2:1b-instruct-fp16

Once the model has finished downloading, we initialize it in LangChain using the ChatOllama class:

In [1]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv(dotenv_path='.env')

os.environ["LANGCHAIN_TRACING_V2"] = os.getenv("LANGSMITH_TRACING")
os.environ["LANGCHAIN_ENDPOINT"] = os.getenv("LANGSMITH_ENDPOINT")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGSMITH_PROJECT")

In [7]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.2:1b"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)

# initialize another LLM with temperature 0.9, this makes the LLM more creative
creative_llm = ChatOllama(temperature=0.9, model=model_name)

## Traditional Chains vs LCEL Chains

In this section we're going to dive into a basic example comparing the main difference between the two chains. For this we will use a basic example of finding the user's report, where the user must input a specific topic, and then the LLM will look and return a report on the specified topic.

In [8]:
from langchain.schema.output_parser import StrOutputParser

output_parser = StrOutputParser()

In [9]:
from langchain import PromptTemplate

prompt_template = "Give me a small report of {topic}"

prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_template
)

Here is a standard LLMChain, this has the basic function like properties where we would call a name of the function, and pass in parameters to adjust the function, in this case, prompt, llm and output_parser, where the prompt will be used by the model, and then the result of the model will be used by the output parser.

In [10]:
from langchain.chains import LLMChain

chain = LLMChain(
    prompt = prompt,
    llm = llm,
    output_parser = output_parser
)

In [11]:
result = chain.invoke("AI")
print(result)

{'topic': 'AI', 'text': "Here's a brief report on Artificial Intelligence (AI):\n\n**What is Artificial Intelligence?**\n\nArtificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception. AI systems use algorithms and data to make decisions, learn from experience, and improve their performance over time.\n\n**Types of AI:**\n\n1. **Narrow or Weak AI:** Designed to perform a specific task, such as facial recognition, language translation, or playing chess.\n2. **General or Strong AI:** A hypothetical AI system that can perform any intellectual task that a human can, with no limitations.\n3. **Superintelligence:** An AI system that is significantly more intelligent than the best human minds.\n\n**AI Techniques:**\n\n1. **Machine Learning (ML):** A subset of AI that enables systems to learn from data and improve their performance over time.\n2. **

This is a LCEL chain, as you can see, this initially appears to be abit 'hacky' but the abstraction allows for us to skip calling a function and pass our variables into eachother instead.

In [12]:
lcel_chain = prompt | llm | output_parser

In [13]:
result = lcel_chain.invoke("AI")
print(result)

Here's a brief report on Artificial Intelligence (AI):

**What is Artificial Intelligence?**

Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception. AI systems use algorithms and data to make decisions, learn from experience, and improve their performance over time.

**Types of AI:**

1. **Narrow or Weak AI:** Designed to perform a specific task, such as facial recognition, language translation, or playing chess.
2. **General or Strong AI:** A hypothetical AI system that can perform any intellectual task that a human can, with no limitations.
3. **Superintelligence:** An AI system that is significantly more intelligent than the best human minds.

**AI Techniques:**

1. **Machine Learning (ML):** A subset of AI that enables systems to learn from data and improve their performance over time.
2. **Deep Learning (DL):** A type of ML that 

## How Does LCEL Work?

The concept is reasonably simple, you start on the far left side of the line, look at the first variable, and the output to that variable is passed into the next variable, before we had ***prompt | llm | output_parser***, we can see that the prompt, feeds into the model, then the model result feeds into the output parser.

When we use the pipe operator **|** we are basically looking for a or function, this is where we can find a chained functionallity to the variable.

Let's make a basic runnable class to show you the basics of how this works.

In [14]:
class Runnable:
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        def chained_func(*args, **kwargs):
            return other(self.func(*args, **kwargs))
        return Runnable(chained_func)
    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

We firstly want to make a bunch of random functions, in this case we will do some simple maths with each function.

In [15]:
def add_five(x):
    return x+5
def sub_five(x):
    return x-5
def mul_five(x):
    return x*5

Now we want to coat our functions with the runnable so that the or function can be identified.

In [16]:
add_five = Runnable(add_five)
sub_five = Runnable(sub_five)
mul_five = Runnable(mul_five)

Now as you can see we can chain together the 3 functions we made just earlier using the or function, and if we switch the or functions out for the pipe operator, it does exactly the same.

In [17]:
chain = (add_five).__or__(sub_five).__or__(mul_five)
chain(3)

15

In [18]:
chain = add_five | sub_five | mul_five
chain(3)

15

## LCEL Parallel Use-Case Scenario

In this section we will go over how we can use LCEL's parallel capabilities.

To start us of with, we will have two statements, one side telling the AI the month and day Josh was born, and the other telling the AI the year Josh was born. We will then embed the statements and feed them into AI together.

In [21]:
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch

embedding = OllamaEmbeddings()
vecstore_a = DocArrayInMemorySearch.from_texts(
    ["half the info is here", "Joshs' birthday is June the 12th"], 
    embedding=embedding
)
vecstore_b = DocArrayInMemorySearch.from_texts(
    ["the other half of the info is here", "Josh was born in 2002"], 
    embedding=embedding
)

Here you can see the prompt does have three inputs, two for context and one for the question itself.

In [22]:
prompt_str = """ using the following context answer the question
Context: 
{context_a}
{context_b}

Question:
{question}

Answer: """

In [23]:
from langchain.prompts.chat import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(prompt_str)

Here we are wrapping our vector stores as retrievers so they can be fitted into one big retrieval variable to be used by the prompt.

In [24]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()

retrieval = RunnableParallel(
    {
        "context_a": retriever_a, "context_b": retriever_b, "question": RunnablePassthrough()
    }
)

In [25]:
chain = retrieval | prompt | llm | output_parser

In [26]:
result = chain.invoke("What was the date when Josh was born")

In [27]:
result

'Based on the provided context, we can infer that the information about Josh\'s birthdate is split between two documents. \n\nThe first document states that "half the info is here", which implies that it contains only one part of the information about Josh\'s birthdate.\n\nThe second document explicitly states that "the other half of the info is here", which means it contains the complete information about Josh\'s birthdate.\n\nTherefore, we can conclude that the date when Josh was born is mentioned in the first document.'

## LangChain's RunnableLambdas

You can use arbitrary functions as Runnables. This is useful for formatting or when you need functionality not provided by other LangChain components, and custom functions used as Runnables are called RunnableLambdas.

Note that all inputs to these functions need to be a SINGLE argument. If you have a function that accepts multiple arguments, you should write a wrapper that accepts a single dict input and unpacks it into multiple arguments.

In [28]:
from langchain_core.runnables import RunnableLambda

Here we can make some custom functions that do simple maths again, and see that RunnableLambdas can compile and output the correct results.

In [29]:
def add_five(x):
    return x+5
def sub_five(x):
    return x-5
def mul_five(x):
    return x*5

In [30]:
add_five = RunnableLambda(add_five)
sub_five = RunnableLambda(sub_five)
mul_five = RunnableLambda(mul_five)

In [31]:
chain = add_five | sub_five | mul_five

In [32]:
chain.invoke(3)

15

Now we want to try something a little more testing, so this time we will generate a report, and we will try and edit that report using this functionallity.

In [33]:
prompt_str = "give me a small report about {topic}"
prompt = PromptTemplate(
    input_variables=["topic"],
    template=prompt_str
)

In [34]:
chain = prompt | llm | output_parser

In [35]:
result = chain.invoke("AI")

In [36]:
print(result)

Here's a brief report on Artificial Intelligence (AI):

**What is Artificial Intelligence?**

Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception. AI systems use algorithms and data to make decisions, identify patterns, and learn from experience.

**Types of AI:**

1. **Narrow or Weak AI:** Designed to perform a specific task, such as facial recognition, language translation, or playing chess.
2. **General or Strong AI:** A hypothetical AI system that can perform any intellectual task that humans can, with no limitations.
3. **Superintelligence:** An AI system that is significantly more intelligent than the best human minds.

**AI Applications:**

1. **Virtual Assistants:** Siri, Alexa, and Google Assistant use AI to understand voice commands and respond accordingly.
2. **Image Recognition:** AI-powered systems like Facebook's facia

Here we are making two functions, one that will get rid of the introduction to the AI finding the information, instead we will just see the information, and word replacer that will replace AI with Josh!

In [37]:
def extract_fact(x):
    if "\n\n" in x:
        return "\n".join(x.split("\n\n")[1:])
    else:
        return x
    
old_word = "AI"
new_word = "Josh"

def replace_word(x):
    return x.replace(old_word, new_word)

Lets wrap these functions and see what the output is!

In [38]:
extract_fact = RunnableLambda(extract_fact)
replace_word = RunnableLambda(replace_word)

In [39]:
chain = prompt | llm | output_parser | extract_fact | replace_word

In [40]:
result = chain.invoke("AI")

In [41]:
print(result)

**What is Artificial Intelligence?**
Artificial Intelligence (Josh) refers to the development of computer systems that can perform tasks that typically require human intelligence, such as learning, problem-solving, decision-making, and perception. Josh systems use algorithms and data to make decisions, identify patterns, and learn from experience.
**Types of Josh:**
1. **Narrow or Weak Josh:** Designed to perform a specific task, such as facial recognition, language translation, or playing chess.
2. **General or Strong Josh:** A hypothetical Josh system that can perform any intellectual task that humans can, with no limitations.
3. **Superintelligence:** An Josh system that is significantly more intelligent than the best human minds.
**Josh Applications:**
1. **Virtual Assistants:** Siri, Alexa, and Google Assistant use Josh to understand voice commands and respond accordingly.
2. **Image Recognition:** Josh-powered systems like Facebook's facial recognition feature and Google Photos' 